# Regression
Let's go back to our weather dataset. Instead of the binary `BBQ_weather` label we could try to predict a continuous value.

Let's go back to our original dataset

In [15]:
import seaborn as sns
import pandas as pd

In [18]:
%ls data

weather_prediction_dataset.csv  weather_test.csv  weather_train.csv


In [19]:
data = pd.read_csv('data/weather_prediction_dataset.csv')
data.head()

,DATE,MONTH,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,...,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,TOURS_humidity,TOURS_pressure,TOURS_global_radiation,TOURS_precipitation,TOURS_temp_mean,TOURS_temp_min,TOURS_temp_max
0,20000101,1,8,0.89,1.0286,0.20,0.03,0.0,2.9,1.6,...,-15.2,-17.0,-13.4,0.97,1.0275,0.25,0.04,8.5,7.2,9.8
1,20000102,1,8,0.87,1.0318,0.25,0.00,0.0,3.6,2.7,...,-13.7,-15.0,-12.3,0.99,1.0293,0.17,0.16,7.9,6.6,9.2
2,20000103,1,5,0.81,1.0314,0.50,0.00,3.7,2.2,0.1,...,-9.2,-12.5,-5.8,0.91,1.0267,0.27,0.00,8.1,6.6,9.6
3,20000104,1,7,0.79,1.0262,0.63,0.35,6.9,3.9,0.5,...,-5.6,-7.0,-4.2,0.95,1.0222,0.11,0.44,8.6,6.4,10.8
4,20000105,1,5,0.90,1.0246,0.51,0.07,3.7,6.0,3.8,...,-7.6,-9.4,-5.8,0.95,1.0209,0.39,0.04,8.0,6.4,9.5


Let us try to predict the number of sunshine hours in Basel based in the previous day. We will have to create the new labels.

We will have to select our 3 years of data again. Then we create a new column that will show the number of sunshine hours in Basel on the next day.

In [24]:
n_rows = 365*3
weather_3years = data[:n_rows].drop(columns=['DATE'])
sunshine_tomorrow = data[1:n_rows + 1]['BASEL_sunshine']

sunshine_tomorrow.describe()

count    1095.000000
mean        4.531781
std         4.423493
min         0.000000
25%         0.400000
50%         3.400000
75%         7.850000
max        15.300000
Name: BASEL_sunshine, dtype: float64

We will have to split our training and test set again:

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(weather_3years, sunshine_tomorrow, test_size=0.3, random_state=0)

In [29]:
y_train

182     9.6
426     4.1
1022    2.0
10      0.0
857     7.2
       ... 
1034    4.2
764     7.8
836     2.1
560     0.6
685     3.6
Name: BASEL_sunshine, Length: 766, dtype: float64